In [1]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
import os
from astropy import wcs
from astropy.table import Table, hstack, Column
from astropy.utils.console import ProgressBar
from astropy.table import Table, hstack, Column
import pandas as pd

In [12]:
### load constants and functions
# Image information and assumptions
#Temp            = 20.0
Wave            = (3.0e8/226.e9)
Wave0           = 1.3e-3
k0              = 0.899
nu              = 3.e08/Wave
nu0             = 3.e08/Wave0
beta            = 1.75
Kappag2d        = k0*((nu/nu0)**beta)
g2d             = 100.0
Kappa           = Kappag2d / g2d
mu              = 2.8 # express everything in H2

# Constants
G = 6.67408e-11
msun = 1.989e33
mh = 1.6737236e-27
pc2cm = 3.08567758e18
as2persr = 4.25e10 
percm2perm = 1.0e6
JyperSr_to_JyperPix = (3.0462*(10**-4))*((0.00013888888888)**2)
JyperSr_to_Jyperassqr = ((np.pi*180.)**(-2))*(3600**(-1))
hplanck = 6.63e-34
clight = 2.99792e8
kboltzmann = 1.381e-23
sin1yr = 3.15569e7


In [13]:
# def calc_Tb(flux_jysr,freq_hz):
#     return (flux_jysr*clight*clight)/(2*kboltzmann*freq_hz**2)

# def calc_Tb(flux_jysr,freq_GHz,ba_sr):
#     return 1.222e3 * (np.pi/(4*np.log(2))) * (flux_jysr/(ba_sr*freq_GHz*freq_GHz))

def calc_Tb(flux_MJySr,wl_um):
    return (1e-26*1e6*flux_MJySr*(1e-6*wl_um)**2)/(2*1.38e-23)



In [78]:
flux_MJySr = 6774
ba_Sr = 3.51e-8
# print(calc_Tb(flux_jysr,572.5,ba_sr))
print(calc_Tb(flux_MJySr,523.7))

0.6731335826304348


In [15]:
hco_tab = Table.read('/Users/hph/Documents/SOFIA_cycle8/asu.fit')

In [16]:
inds = np.where(np.array(hco_tab['HCO'])==3)
print(len(inds[0]))

80


In [17]:
hco_subtab = hco_tab[np.array(hco_tab['HCO'])==3]
for i in range(80): print(hco_subtab['Seq'][i])

1363
1377
1398
1412
1421
1466
1491
1518
1521
1584
1659
1780
1796
1803
1833
1869
1943
1956
2019
2072
2081
2097
2153
2165
2175
2314
2382
2388
2561
2971
3018
3116
3184
3202
3210
3284
3313
3322
3383
3507
3594
3767
3777
3807
3829
3913
3938
3998
4055
4152
4472
4509
4518
4526
4527
4539
4547
4560
4916
5120
5252
5341
5414
5433
5530
5641
5700
5798
5850
5926
5960
5973
6116
6159
6313
6909
7235
7364
7380
7493


In [67]:
(clight / 572.5e9)*1e6

523.6541484716157

In [73]:
cond = (hco_subtab['Seq']==4518)
print(hco_subtab['RAJ2000'][cond])
print(hco_subtab['DEJ2000'][cond])

 RAJ2000  
 "h:m:s"  
----------
18 47 41.4
  DEJ2000  
  "d:m:s"  
-----------
-02 00 20.7


In [45]:
hco_subtab.keys()

['recno',
 'Seq',
 'A',
 'BGPS',
 'RAJ2000',
 'DEJ2000',
 'Cat',
 'HCO',
 'vLSR',
 'Tmb',
 'e_Tmb',
 'I',
 'e_I',
 'Delv',
 'e_Delv',
 'FWZI',
 'pPap',
 'Simbad']

In [71]:
hco_subtab

recno,Seq,A,BGPS,RAJ2000,DEJ2000,Cat,HCO,vLSR,Tmb,e_Tmb,I,e_I,Delv,e_Delv,FWZI,pPap,Simbad
,,,,"""h:m:s""","""d:m:s""",,,km / s,K,K,K km / s,K km / s,km / s,km / s,km / s,,
int32,int32,str1,str15,str10,str11,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,int32,str7
57,1363,*,G008.458-00.224,18 05 22.6,-21 44 43.9,2,3,36.4,1.486,0.075,9.273,0.338,0.0,0.0,9.0,1,Simbad
71,1377,*,G008.670-00.356,18 06 19.1,-21 37 30.4,2,3,33.1,5.172,0.042,34.999,0.253,0.0,0.0,22.4,1,Simbad
92,1398,*,G008.872-00.318,18 06 36.0,-21 25 48.9,2,3,40.9,0.362,0.053,2.83,0.208,0.0,0.0,9.0,1,Simbad
106,1412,*,G009.212-00.202,18 06 52.6,-21 04 36.8,2,3,40.9,1.016,0.042,5.982,0.189,0.0,0.0,11.2,1,Simbad
115,1421,*,G009.620+00.194,18 06 14.9,-20 31 39.2,2,3,6.2,7.393,0.03,52.18,0.187,0.0,0.0,30.3,1,Simbad
160,1466,*,G010.214-00.324,18 09 24.6,-20 15 39.2,1,3,9.4,0.983,0.03,5.151,0.09,0.0,0.0,11.2,2,Simbad
185,1491,*,G010.416-00.030,18 08 44.1,-19 56 28.6,2,3,67.8,0.24,0.037,1.088,0.134,0.0,0.0,4.5,1,Simbad
212,1518,*,G010.681-00.028,18 09 16.2,-19 42 29.3,1,3,50.2,1.113,0.035,3.66,0.091,0.0,0.0,9.0,2,Simbad
